### 0. Install YOLOv8

In [1]:
!pip install ultralytics

In [2]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()
!yolo checks



Ultralytics YOLOv8.2.95 🚀 Python-3.12.4 torch-2.4.1+cpu CPU (Intel Core(TM) i7-10750H 2.60GHz)
Setup complete ✅ (12 CPUs, 31.9 GB RAM, 334.0/475.6 GB disk)

OS                  Windows-11-10.0.22631-SP0
Environment         Windows
Python              3.12.4
Install             pip
RAM                 31.91 GB
CPU                 Intel Core(TM) i7-10750H 2.60GHz
CUDA                None

numpy               ✅ 1.26.4<2.0.0,>=1.23.0
matplotlib          ✅ 3.8.4>=3.3.0
opencv-python       ✅ 4.10.0.84>=4.6.0
pillow              ✅ 10.3.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.32.2>=2.23.0
scipy               ✅ 1.13.1>=1.4.1
torch               ✅ 2.4.1>=1.8.0
torchvision         ✅ 0.19.1>=0.9.0
tqdm                ✅ 4.66.4>=4.64.0
psutil              ✅ 5.9.0
py-cpuinfo          ✅ 9.0.0
pandas              ✅ 2.2.2>=1.1.4
seaborn             ✅ 0.13.2>=0.11.0
ultralytics-thop    ✅ 2.0.6>=2.0.0
torch               ✅ 2.4.1!=2.4.0,>=1.8.0; sys_platform == "win32"


### 0.5. Create Dataset

In [3]:
import os
import shutil
import random
import xml.etree.ElementTree as ET

def create_dataset_yolo():
    counter = 0
    root_dir = '../Manga109'
    images_dir = os.path.join(root_dir, 'images')
    annotations_dir = os.path.join(root_dir, 'annotations')

    # Create train, val, test directories
    for dataset_type in ['train', 'val', 'test']:
        os.makedirs(f'../Manga109_YOLO/{dataset_type}/images', exist_ok=True)
        os.makedirs(f'../Manga109_YOLO/{dataset_type}/labels', exist_ok=True)

    for annotation_file in os.listdir(annotations_dir):
        tree = ET.parse(os.path.join(annotations_dir, annotation_file))
        root = tree.getroot()
        book_title = root.get('title')

        for page in root.findall('.//page'):
            page_index = page.get('index')
            image_path = os.path.join(images_dir, book_title, f'{int(page_index):03}.jpg')

            # Randomly assign to train/val/test and copy image + txt file to new dir
            dataset_type = random.choices(['train', 'val', 'test'], weights=[0.85, 0.1, 0.05])[0]

            # Copy the image to the new directory
            shutil.copyfile(image_path, f'../Manga109_YOLO/{dataset_type}/images/{book_title}_{int(page_index):03}.jpg')

            # Create a new text file in the labels directory for that chosen dataset
            label_file_path = f'../Manga109_YOLO/{dataset_type}/labels/{book_title}_{int(page_index):03}.txt'
            label_file = open(label_file_path, 'w')

            # populate text file with annotations
            width = int(page.get('width'))
            height = int(page.get('height'))

            for i, frame_type in enumerate(['face', 'body', 'text', 'frame']):
                for frame in page.findall(frame_type):
                    bbox = (
                        (int(frame.get('xmin')) + int(frame.get('xmax'))) / (2 * width),
                        (int(frame.get('ymin')) + int(frame.get('ymax'))) / (2 * height),
                        (int(frame.get('xmax')) - int(frame.get('xmin'))) / width,
                        (int(frame.get('ymax')) - int(frame.get('ymin'))) / height,
                    )
                    label_file.write(f'{i} {" ".join(map(str, bbox))}\n')
            label_file.close()

            counter += 1  # Increment the counter

            # Print the counter every 50 increments
            if counter % 50 == 0:
                print(f'Processed {counter} images')
    return 0

def main():
    create_dataset_yolo()
    return 0

if __name__ == '__main__':
    main()

Processed 50 images
Processed 100 images
Processed 150 images
Processed 200 images
Processed 250 images
Processed 300 images
Processed 350 images
Processed 400 images
Processed 450 images
Processed 500 images
Processed 550 images
Processed 600 images
Processed 650 images
Processed 700 images
Processed 750 images
Processed 800 images
Processed 850 images
Processed 900 images
Processed 950 images
Processed 1000 images
Processed 1050 images
Processed 1100 images
Processed 1150 images
Processed 1200 images
Processed 1250 images
Processed 1300 images
Processed 1350 images
Processed 1400 images
Processed 1450 images
Processed 1500 images
Processed 1550 images
Processed 1600 images
Processed 1650 images
Processed 1700 images
Processed 1750 images
Processed 1800 images
Processed 1850 images
Processed 1900 images
Processed 1950 images
Processed 2000 images
Processed 2050 images
Processed 2100 images
Processed 2150 images
Processed 2200 images
Processed 2250 images
Processed 2300 images
Processe

### 1. Train Dataset

In [4]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
!yolo task=detect mode=train model=yolov8n.pt data=manga109.yaml epochs=10

^C
